<a href="https://colab.research.google.com/github/lena-33/LenaThabet/blob/main/SolarCastGaza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import requests
import json

api_key = "97dd654d1d70f121b6bf328fd5e855e2"
city = "Gaza, PS"
url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}&units=metric"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    # طباعة ملخص بيانات الطقس لليوم الأول
    print("Weather forecast for Gaza (next 5 days, every 3 hours):")
    for entry in data['list'][:8]:  # أول 8 نقاط زمنية (24 ساعة)
        dt_txt = entry['dt_txt']
        temp = entry['main']['temp']
        weather_desc = entry['weather'][0]['description']
        print(f"{dt_txt} - Temp: {temp}°C, Weather: {weather_desc}")
else:
    print("Error fetching data:", response.status_code)
    print(response.text)

Weather forecast for Gaza (next 5 days, every 3 hours):
2025-06-15 15:00:00 - Temp: 27.1°C, Weather: clear sky
2025-06-15 18:00:00 - Temp: 26.15°C, Weather: clear sky
2025-06-15 21:00:00 - Temp: 24.26°C, Weather: clear sky
2025-06-16 00:00:00 - Temp: 21.95°C, Weather: few clouds
2025-06-16 03:00:00 - Temp: 22.68°C, Weather: broken clouds
2025-06-16 06:00:00 - Temp: 24.98°C, Weather: scattered clouds
2025-06-16 09:00:00 - Temp: 26.4°C, Weather: few clouds
2025-06-16 12:00:00 - Temp: 26.59°C, Weather: clear sky


In [13]:
import requests
import pandas as pd

# إعدادات
api_key = "97dd654d1d70f121b6bf328fd5e855e2" # Replace with your actual API key
city = "Gaza,PS"
url = f"https://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}&units=metric"

# بيانات الطقس
response = requests.get(url)

if response.status_code == 200:
    data = response.json()

    # عدد الألواح وقدرة كل واحد
    num_panels = 4
    panel_watt = 300  # 300 واط لكل لوح

    # معامل إنتاج حسب حالة الطقس
    weather_effect = {
        "clear sky": 1.0,
        "few clouds": 0.8,
        "scattered clouds": 0.6,
        "broken clouds": 0.4,
        "overcast clouds": 0.2,
        "light rain": 0.2,
        "moderate rain": 0.2
    }

    # استخراج البيانات إلى DataFrame
    rows = []
    for item in data["list"]:
        time = item["dt_txt"]
        temp = item["main"]["temp"]
        weather = item["weather"][0]["description"]

        hour = int(time[11:13])
        if 6 <= hour <= 18:  # فقط ساعات النهار
            effect = weather_effect.get(weather, 0.5)  # قيمة افتراضية
            energy_kwh = (panel_watt * num_panels / 1000) * effect * 3  # 3 ساعات لكل فترة
            rows.append([time, temp, weather, effect, round(energy_kwh, 2)])

    # إنشاء DataFrame
    df = pd.DataFrame(rows, columns=["Time", "Temp (°C)", "Weather", "Effect", "Estimated kWh"])
    df["Day"] = pd.to_datetime(df["Time"]).dt.date

    # تجميع الطاقة حسب اليوم
    daily_summary = df.groupby("Day")["Estimated kWh"].sum().reset_index()
    daily_summary.columns = ["Day", "Total Energy (kWh)"]

    print("\n🔋 تقدير الطاقة المنتجة يوميًا (عدد الألواح:", num_panels, ")")
    print(daily_summary)
else:
    print("Error fetching data:", response.status_code)
    print(response.text)


🔋 تقدير الطاقة المنتجة يوميًا (عدد الألواح: 4 )
          Day  Total Energy (kWh)
0  2025-06-15                7.20
1  2025-06-16               15.84
2  2025-06-17               18.00
3  2025-06-18               18.00
4  2025-06-19               18.00
5  2025-06-20               10.80


In [15]:
!pip install gradio --quiet

In [ ]:
import gradio as gr
import requests
import pandas as pd
from datetime import datetime

def solar_prediction(num_panels):
    # إعدادات الطقس
    api_key = "97dd654d1d70f121b6bf328fd5e855e2"
    city = "Gaza,PS"
    url = f"https://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    data = response.json()

    panel_watt = 300  # واط لكل لوح

    weather_effect = {
        "clear sky": 1.0,
        "few clouds": 0.8,
        "scattered clouds": 0.6,
        "broken clouds": 0.4,
        "overcast clouds": 0.2,
        "light rain": 0.2,
        "moderate rain": 0.2
    }

    # جمع البيانات
    rows = []
    for item in data["list"]:
        time = item["dt_txt"]
        temp = item["main"]["temp"]
        weather = item["weather"][0]["description"]

        hour = int(time[11:13])
        if 6 <= hour <= 18:
            effect = weather_effect.get(weather, 0.5)
            energy_kwh = (panel_watt * num_panels / 1000) * effect * 3
            rows.append([time, temp, weather, effect, round(energy_kwh, 2)])

    df = pd.DataFrame(rows, columns=["Time", "Temp (°C)", "Weather", "Effect", "Estimated kWh"])
    df["Day"] = pd.to_datetime(df["Time"]).dt.date
    daily_summary = df.groupby("Day")["Estimated kWh"].sum().reset_index()
    daily_summary.columns = ["Day", "Total Energy (kWh)"]

    # إضافة رسائل ذكية
    def get_message(kwh):
        if kwh >= 17:
            return "☀☀ يوم ممتاز للشحن الكامل"
        elif kwh >= 13:
            return "☀ يوم جيد جدًا للشحن"
        elif kwh >= 9:
            return "🌤 إنتاج جيد، استغل النهار"
        else:
            return "🌥 إنتاج منخفض – جدول شحنك بعناية"

    daily_summary["Message"] = daily_summary["Total Energy (kWh)"].apply(get_message)

    return daily_summary

# واجهة Gradio
gr.Interface(
    fn=solar_prediction,
    inputs=gr.Number(label="🔢 عدد الألواح الشمسية", value=4),
    outputs=gr.Dataframe(label="🔋 توقع إنتاج الطاقة (لـ 5 أيام)"),
    title="☀ توقع إنتاج الكهرباء من الطاقة الشمسية",
    description="أدخل عدد الألواح الشمسية لتقدير كمية الطاقة المنتجة في الأيام الخمسة القادمة في غزة"
).launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b7eec8ee57f26059e8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Created dataset file at: .gradio/flagged/dataset1.csv
